# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 16 2022 12:36PM,244280,12,HH-36428,Hush Hush,Released
1,Aug 16 2022 12:36PM,244280,12,HH-36429,Hush Hush,Released
2,Aug 16 2022 12:36PM,244280,12,HH-36431,Hush Hush,Released
3,Aug 16 2022 12:36PM,244280,12,HH-36432,Hush Hush,Released
4,Aug 16 2022 12:36PM,244280,12,HH-36433,Hush Hush,Released
5,Aug 16 2022 12:33PM,244279,10,8409929,MedStone Pharma LLC,Released
6,Aug 16 2022 12:33PM,244279,10,8409930,MedStone Pharma LLC,Released
7,Aug 16 2022 12:33PM,244279,10,8409931,MedStone Pharma LLC,Released
8,Aug 16 2022 12:33PM,244279,10,8409932,MedStone Pharma LLC,Released
9,Aug 16 2022 12:33PM,244279,10,8409933,MedStone Pharma LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,244277,Released,7
44,244278,Executing,2
45,244278,Released,3
46,244279,Released,6
47,244280,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244276,NaN,1.0,NaN
244277,NaN,NaN,7.0
244278,NaN,2.0,3.0
244279,NaN,NaN,6.0
244280,NaN,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244164,0.0,0.0,1.0
244183,0.0,0.0,1.0
244189,0.0,0.0,1.0
244191,0.0,0.0,1.0
244192,0.0,0.0,12.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244164,0,0,1
244183,0,0,1
244189,0,0,1
244191,0,0,1
244192,0,0,12


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244164,0,0,1
1,244183,0,0,1
2,244189,0,0,1
3,244191,0,0,1
4,244192,0,0,12


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244164,,,1
1,244183,,,1
2,244189,,,1
3,244191,,,1
4,244192,,,12


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 16 2022 12:36PM,244280,12,Hush Hush
5,Aug 16 2022 12:33PM,244279,10,MedStone Pharma LLC
11,Aug 16 2022 12:31PM,244277,10,"Methapharm, Inc."
18,Aug 16 2022 12:24PM,244275,16,Seqirus
19,Aug 16 2022 12:20PM,244278,19,"AdvaGen Pharma, Ltd"
24,Aug 16 2022 12:18PM,244276,102,"Senseonics, Incorporated"
25,Aug 16 2022 12:16PM,244274,16,Seqirus
26,Aug 16 2022 12:00PM,244271,19,"GUSA Granules USA, Inc."
27,Aug 16 2022 11:54AM,244270,19,"GUSA Granules USA, Inc."
28,Aug 16 2022 11:52AM,244267,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 16 2022 12:36PM,244280,12,Hush Hush,,,5
1,Aug 16 2022 12:33PM,244279,10,MedStone Pharma LLC,,,6
2,Aug 16 2022 12:31PM,244277,10,"Methapharm, Inc.",,,7
3,Aug 16 2022 12:24PM,244275,16,Seqirus,,,1
4,Aug 16 2022 12:20PM,244278,19,"AdvaGen Pharma, Ltd",,2,3
5,Aug 16 2022 12:18PM,244276,102,"Senseonics, Incorporated",,1,
6,Aug 16 2022 12:16PM,244274,16,Seqirus,,,1
7,Aug 16 2022 12:00PM,244271,19,"GUSA Granules USA, Inc.",,,1
8,Aug 16 2022 11:54AM,244270,19,"GUSA Granules USA, Inc.",,,1
9,Aug 16 2022 11:52AM,244267,10,ISDIN Corporation,,,18


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 12:36PM,244280,12,Hush Hush,5,,
1,Aug 16 2022 12:33PM,244279,10,MedStone Pharma LLC,6,,
2,Aug 16 2022 12:31PM,244277,10,"Methapharm, Inc.",7,,
3,Aug 16 2022 12:24PM,244275,16,Seqirus,1,,
4,Aug 16 2022 12:20PM,244278,19,"AdvaGen Pharma, Ltd",3,2,
5,Aug 16 2022 12:18PM,244276,102,"Senseonics, Incorporated",,1,
6,Aug 16 2022 12:16PM,244274,16,Seqirus,1,,
7,Aug 16 2022 12:00PM,244271,19,"GUSA Granules USA, Inc.",1,,
8,Aug 16 2022 11:54AM,244270,19,"GUSA Granules USA, Inc.",1,,
9,Aug 16 2022 11:52AM,244267,10,ISDIN Corporation,18,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 12:36PM,244280,12,Hush Hush,5,,
1,Aug 16 2022 12:33PM,244279,10,MedStone Pharma LLC,6,,
2,Aug 16 2022 12:31PM,244277,10,"Methapharm, Inc.",7,,
3,Aug 16 2022 12:24PM,244275,16,Seqirus,1,,
4,Aug 16 2022 12:20PM,244278,19,"AdvaGen Pharma, Ltd",3,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 12:36PM,244280,12,Hush Hush,5.0,NaN,NaN
1,Aug 16 2022 12:33PM,244279,10,MedStone Pharma LLC,6.0,NaN,NaN
2,Aug 16 2022 12:31PM,244277,10,"Methapharm, Inc.",7.0,NaN,NaN
3,Aug 16 2022 12:24PM,244275,16,Seqirus,1.0,NaN,NaN
4,Aug 16 2022 12:20PM,244278,19,"AdvaGen Pharma, Ltd",3.0,2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 12:36PM,244280,12,Hush Hush,5.0,0.0,0.0
1,Aug 16 2022 12:33PM,244279,10,MedStone Pharma LLC,6.0,0.0,0.0
2,Aug 16 2022 12:31PM,244277,10,"Methapharm, Inc.",7.0,0.0,0.0
3,Aug 16 2022 12:24PM,244275,16,Seqirus,1.0,0.0,0.0
4,Aug 16 2022 12:20PM,244278,19,"AdvaGen Pharma, Ltd",3.0,2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3175229,102.0,0.0,0.0
102,244276,0.0,1.0,0.0
12,244280,5.0,0.0,0.0
15,732653,46.0,33.0,45.0
16,1465406,6.0,0.0,0.0
19,1465505,8.0,3.0,0.0
20,1465437,61.0,0.0,0.0
21,732583,3.0,0.0,0.0
22,732772,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3175229,102.0,0.0,0.0
1,102,244276,0.0,1.0,0.0
2,12,244280,5.0,0.0,0.0
3,15,732653,46.0,33.0,45.0
4,16,1465406,6.0,0.0,0.0
5,19,1465505,8.0,3.0,0.0
6,20,1465437,61.0,0.0,0.0
7,21,732583,3.0,0.0,0.0
8,22,732772,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,102.0,0.0,0.0
1,102,0.0,1.0,0.0
2,12,5.0,0.0,0.0
3,15,46.0,33.0,45.0
4,16,6.0,0.0,0.0
5,19,8.0,3.0,0.0
6,20,61.0,0.0,0.0
7,21,3.0,0.0,0.0
8,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,102.0
1,102,Released,0.0
2,12,Released,5.0
3,15,Released,46.0
4,16,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,19,20,21,22
Status,,,,,,,,,
Completed,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0
Executing,0.0,1.0,0.0,33.0,0.0,3.0,0.0,0.0,0.0
Released,102.0,0.0,5.0,46.0,6.0,8.0,61.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,1.0,0.0,33.0,0.0,3.0,0.0,0.0,0.0
2,Released,102.0,0.0,5.0,46.0,6.0,8.0,61.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,1.0,0.0,33.0,0.0,3.0,0.0,0.0,0.0
2,Released,102.0,0.0,5.0,46.0,6.0,8.0,61.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()